# Download embeddings model from hugging face

In [ ]:

from huggingface_hub import hf_hub_download
import torch
from model import CBOW

# model withou title heckers 
model_path = hf_hub_download(repo_id="cocoritzy/cbow-upvotes_model", filename="cbow_model.pt")


In [ ]:
# These must match what we trained with
input_dim = 100
# model = CBOW(vocab_size=30000, embedding_dim=input_dim)
vocab_size = xx

model = CBOW(vocab_size=vocab_size, embedding_dim=input_dim) 

# Load model weights, saving to gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(device)

model.eval()

# Load vocab mappings
token_to_index = checkpoint["token_to_index"]

# embedding_layer = model.embeddings
embedding_layer = model.embedding

# Load query and title documents 

In [ ]:

from datasets import load_dataset

dataset = load_dataset("cocoritzy/week_2_triplet_dataset_hard_negatives")

split_data = dataset["train"].train_test_split(test_size=0.2, seed=42) # 80% train, 20% test

# Convert title to embedding

In [ ]:

def title_to_embedding(title):
    tokens = title.lower().split()
    indices = [token_to_index.get(tok, 0) for tok in tokens]  # 0 for unknowns

    indices_tensor = torch.tensor(indices, dtype=torch.long, device=device)

    with torch.no_grad():
        embeds = embedding_layer(indices_tensor) # [num_tokens, embedding_dim]
        return embeds.mean(dim=0) # average pooling